In [1]:
import uproot
import torch

In [2]:
from torch_geometric.data import Data
from torch_geometric.data import InMemoryDataset
from graph_utils import load_tree


class HGCALTracksters(InMemoryDataset):
    
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['tracksters_ds_10e.root']

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):

        filename = f"{self.root}/{self.raw_file_names[0]}"
        tracksters = uproot.open({filename: "tracksters"})

        dataset = []

        for g, label in load_tree(tracksters):
            x = torch.tensor([pos for _, pos in g.nodes("pos")])
            edge_index = torch.tensor(list(g.edges())).T
            y = torch.tensor(label)
            dataset.append(Data(x, edge_index=edge_index, y=y))

        data, slices = self.collate(dataset)
        torch.save((data, slices), self.processed_paths[0])

In [4]:
import torch_geometric.transforms as T
transform = T.Compose([T.NormalizeFeatures()])

ds = HGCALTracksters("data", transform=transform)

# prepare data
train_set = ds[:2000]
test_set = ds[2000:]

Processing...
Done!


In [5]:
train_set[0]

Data(x=[4, 3], edge_index=[2, 6], y=[1])

In [16]:
print(f"PyTorch version: {torch.__version__}")
# Check PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
# print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
# print(f"Is MPS available? {torch.backends.mps.is_available()}")
# device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else "cpu")

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

PyTorch version: 1.13.0.dev20220622
Using device: cpu


In [12]:
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.loader import DataLoader

In [13]:
class TracksterClassifier(torch.nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(TracksterClassifier, self).__init__(**kwargs)

        self.conv1 = GCNConv(in_channels, out_channels)
        self.conv2 = GCNConv(out_channels, out_channels)
        self.dense = torch.nn.Linear(out_channels, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        x = self.dense(x)
        return torch.sigmoid(x)

In [14]:
loss_obj = torch.nn.BCELoss()

def train(model, loader):
    epoch_loss = 0
    for batch in loader:
        model.train()
        batch = batch.to(device)
        optimizer.zero_grad()
        z = model(batch).reshape(-1)
        loss = loss_obj(z, batch.y.type(torch.float))
        epoch_loss += loss
        loss.backward()
        optimizer.step()
    return float(epoch_loss)

@torch.no_grad()
def test(model, data):
    total = 0
    correct = 0
    for batch in data:
        model.eval()
        prediction = (model(batch).reshape(-1) > 0.5).type(torch.int)
        total += len(prediction) 
        correct += sum(prediction == batch.y)
    return correct / total

In [17]:
model = TracksterClassifier(ds.num_node_features, 64)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
train_loader = DataLoader(train_set, batch_size=8, shuffle=True)
test_loader = DataLoader(test_set, batch_size=8, shuffle=True)

for epoch in range(51):
    loss = train(model, train_loader)
    train_acc = test(model, train_loader)
    test_acc = test(model, test_loader)
    if epoch % 5 == 0:
        print(f'Epoch: {epoch}, loss: {loss:.4f}, train acc: {train_acc:.4f}, test acc: {test_acc:.4f}')

Epoch: 0, loss: 163.7070, train acc: 0.6835, test acc: 0.7732
Epoch: 5, loss: 98.3007, train acc: 0.8290, test acc: 0.8821
Epoch: 10, loss: 78.0473, train acc: 0.8840, test acc: 0.9116
Epoch: 15, loss: 76.3529, train acc: 0.8945, test acc: 0.9070
Epoch: 20, loss: 71.8290, train acc: 0.9020, test acc: 0.8934
Epoch: 25, loss: 72.7565, train acc: 0.9070, test acc: 0.9048
Epoch: 30, loss: 69.6265, train acc: 0.9020, test acc: 0.9116
Epoch: 35, loss: 69.6053, train acc: 0.9110, test acc: 0.8980
Epoch: 40, loss: 67.7216, train acc: 0.8985, test acc: 0.9070
Epoch: 45, loss: 66.8574, train acc: 0.9105, test acc: 0.8934
Epoch: 50, loss: 67.5553, train acc: 0.9130, test acc: 0.9002
